<a href="https://colab.research.google.com/github/amay1212/amay-ml/blob/master/all_rag_techniques/simple_csv_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple RAG (Retrieval-Augmented Generation) System for CSV Files

## Overview

This code implements a basic Retrieval-Augmented Generation (RAG) system for processing and querying CSV documents. The system encodes the document content into a vector store, which can then be queried to retrieve relevant information.

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

## Key Components

1. Loading and spliting csv files.
2. Vector store creation using [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) and OpenAI embeddings
3. Retriever setup for querying the processed documents
4. Creating a question and answer over the csv data.

## Method Details

### Document Preprocessing

1. The csv is loaded using langchain Csvloader
2. The data is split into chunks.


### Vector Store Creation

1. OpenAI embeddings are used to create vector representations of the text chunks.
2. A FAISS vector store is created from these embeddings for efficient similarity search.

### Retriever Setup

1. A retriever is configured to fetch the most relevant chunks for a given query.

## Benefits of this Approach

1. Scalability: Can handle large documents by processing them in chunks.
2. Flexibility: Easy to adjust parameters like chunk size and number of retrieved results.
3. Efficiency: Utilizes FAISS for fast similarity search in high-dimensional spaces.
4. Integration with Advanced NLP: Uses OpenAI embeddings for state-of-the-art text representation.

## Conclusion

This simple RAG system provides a solid foundation for building more complex information retrieval and question-answering systems. By encoding document content into a searchable vector store, it enables efficient retrieval of relevant information in response to queries. This approach is particularly useful for applications requiring quick access to specific information within a csv file.

import libries

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [1]:
# Install required packages
!pip install faiss-cpu langchain langchain-community langchain-openai pandas python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

# # Load environment variables from a .env file
# load_dotenv()

# # Set the OpenAI API key environment variable
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

In [3]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/customers-100.csv https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/customers-100.csv


--2025-09-07 16:26:16--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206372 (202K) [application/octet-stream]
Saving to: ‘data/Understanding_Climate_Change.pdf’

data/Understanding_ 100%[===================>] 201.54K  --.-KB/s    in 0.03s   

2025-09-07 16:26:16 (5.92 MB/s) - ‘data/Understanding_Climate_Change.pdf’ saved [206372/206372]

--2025-09-07 16:26:16--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/customers-100.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:4

In [4]:
import pandas as pd

file_path = ('data/customers-100.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


load and process csv data

In [5]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

In [8]:
docs[-1]

Document(metadata={'source': 'data/customers-100.csv', 'row': 99}, page_content='Index: 100\nCustomer Id: 2354a0E336A91A1\nFirst Name: Clarence\nLast Name: Haynes\nCompany: Le, Nash and Cross\nCity: Judymouth\nCountry: Honduras\nPhone 1: (753)813-6941\nPhone 2: 783.639.1472\nEmail: colleen91@faulkner.biz\nSubscription Date: 2020-03-11\nWebsite: http://www.hatfield-saunders.net/')

Initiate faiss vector store and openai embedding

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-small"
)
index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))


In [15]:
print(len(embeddings.embed_query(" "))) # this outputs the dimension of the embedding model...
vector_store = FAISS(
    embedding_function=embeddings,
    index=index, # 384 dimension indexing,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

384


Add the splitted csv data to the vector store

In [19]:
docs[0]

Document(metadata={'source': 'data/customers-100.csv', 'row': 0}, page_content='Index: 1\nCustomer Id: DD37Cf93aecA6Dc\nFirst Name: Sheryl\nLast Name: Baxter\nCompany: Rasmussen Group\nCity: East Leonard\nCountry: Chile\nPhone 1: 229.077.5154\nPhone 2: 397.884.0519x718\nEmail: zunigavanessa@smith.info\nSubscription Date: 2020-08-24\nWebsite: http://www.stephenson.com/')

In [16]:
vector_store.add_documents(documents=docs)

['8fe26528-6bc4-4c99-b396-b005ab40b2d6',
 'ff3bd8a9-899d-4ebe-ab0b-ecbadafecf4d',
 '4ad5eddc-2417-4c68-8be9-35e1a260799c',
 'ef5cbb03-961b-4a07-83ea-13e0428f8140',
 '2e357a28-2a39-4eb0-b262-ca841a34903b',
 'c09245b7-39a7-4e10-8a4d-2883370a3cfe',
 'cab30616-e2a6-491d-9ad2-7bc13064b464',
 '19fb37ff-5643-4d37-8dd6-1bec2fdd84c9',
 '1a9706bc-7a2a-469e-94fb-9c8a8de19764',
 '77c5dc44-0cad-4ab1-b074-5b1b0496d162',
 'c0def364-2d49-4c67-ab81-b2b310236ed7',
 'cc992509-ad1f-42f2-a0ee-bc20ebb7bae7',
 '94adc382-1d87-4020-9c87-3d84cbe9d839',
 '5f11cc98-918c-468a-a5c8-5ffcf50b9cb2',
 '0ac10ce9-91e1-41e5-82b4-ea4e1d245d4a',
 'b7cafda9-c618-4279-9545-cb9c02512691',
 'f47ea95b-ac9a-4379-826b-c0eb23eb2354',
 '84b0f801-1154-4480-ab5f-35aee745ad18',
 '18c0601e-4813-4164-9aa3-4a3cc7c7bfc6',
 '2564d1b7-2522-416f-86ae-7f612ff89cb1',
 'c4e3fe05-32ca-4d11-982b-9551cd43d2bc',
 'eae3154c-c182-4d39-bd96-a56dfd8c02da',
 'c3bee9ff-a01f-41f7-8a0f-c217d577a565',
 '7e8fce5e-0bac-45ed-82fd-31cba5ce7f4c',
 '459ea8bc-accc-

Create the retrieval chain

In [25]:
vector_store.index.ntotal # total 100 vectors with 384 dimensions are indexed..

100

In [33]:
!pip install langchain_huggingface

In [36]:
model_id = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10
)
hf = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFacePipeline


# llm = HuggingFacePipeline(pipeline=model)  # if `model` is a HuggingFace pipeline

retriever = vector_store.as_retriever()


# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# The {context} placeholder is where retrieved documents will be inserted dynamically at runtime.
# Internally what it does is the following

# 1. User asks a question(query)
# 2. Based on embeddings and vector db we do vector_store.search(top_k=3, query)
# 3. We get some results basically relevant docs page content and we joing it
# 4. This is then passed to the llm model as a `context`.
# 5. This context then helps llm produce the output
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),

])
# print(model) # phi3 mini model
# llm = HuggingFacePipeline(pipeline=model)  # if `model` is a HuggingFace pipeline
# Create the question-answer chain
llm = hf
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

Query the rag bot with a question based on the CSV data

In [40]:
answer= rag_chain.invoke({"input": "which company does sheryl Baxter work for?"})
answer['answer']

"System: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\nIndex: 1\nCustomer Id: DD37Cf93aecA6Dc\nFirst Name: Sheryl\nLast Name: Baxter\nCompany: Rasmussen Group\nCity: East Leonard\nCountry: Chile\nPhone 1: 229.077.5154\nPhone 2: 397.884.0519x718\nEmail: zunigavanessa@smith.info\nSubscription Date: 2020-08-24\nWebsite: http://www.stephenson.com/\n\nIndex: 9\nCustomer Id: C2dE4dEEc489ae0\nFirst Name: Sheryl\nLast Name: Meyers\nCompany: Browning-Simon\nCity: Robersonstad\nCountry: Cyprus\nPhone 1: 854-138-4911x5772\nPhone 2: +1-448-910-2276x729\nEmail: mariokhan@ryan-pope.org\nSubscription Date: 2020-01-13\nWebsite: https://www.bullock.net/\n\nIndex: 49\nCustomer Id: F4Fc91fEAEad286\nFirst Name: Brady\nLast Name: Cohen\nCompany: Osborne-Erickson\nCity: North Eileenville\nCountry: United Arab Emirates\nP

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--simple-csv-rag)